In [1]:
SOURCE_FILE = 'data/posts_data_gab.csv'

import re
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Gab\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
fil_stop_words = []
with open('assets/stop_words_ph.txt') as f:
    for word in f:
        fil_stop_words.append(word.rstrip('\n'))

In [3]:
# Import Dataset
text_data = []
df = pd.read_csv(SOURCE_FILE)
data = df['Body']
data.dropna(inplace=True)

In [4]:
df[0:15]

,Title,Body,Submitted
0,18294,"there’s this guy who’s been making too much malaswa, dagger stares at me, even if I never done anything wrong with him. The way he looked at me is like he’s wants to either beat me up physically or even sexually. he’s too old to act so irrational.\r\n\r\nAteneo pls help make the school a better place rather than bring it a “malaswa playground”","December 9, 2018 2:44:26 AM +08"
1,18293,You call yourself my best friend pero you cant even give 5 mins of your time i needed someone to talk tapos tangina you said stuff pa to make me feel bad about myself\r\n\r\nGet your head out of your fucking ass and stop thinking na everyone wants to make out with u,"December 9, 2018 2:44:05 AM +08"
2,18292,#Daddy Jett 🙌,"December 9, 2018 2:43:23 AM +08"
3,18291,"https://www.facebook.com/braveyawtz/videos/1744836578868329/\r\n\r\nIt's crazy to realize how much all the attention on the 'war on drugs' has died down. Seeing this video was reminiscent to how drug cartels would summarily execute people off the streets. If people are dying left and right, and the notion of labeling their deaths as 'police self defence,' or that the victims fought back, and leaving it at that, then no one is safe to this kind of cruelty.\r\n Tomorrow, the very people being left to die on the streets just might be me and you. Unless of course you're of a rich and/or influential family. Move along, I guess this shouldn't concern you.","December 9, 2018 2:16:15 AM +08"
4,18290,"I saw this girl last thursday. She was hanging out with a bunch of Icans and while I was going down the stairs, I tripped. All of them laughed except for one, she helped me and even asked if I was okay. I was so flustered and just stared at her face. She had the most amazing smile and I couldn't even utter a word. It was funny because she actually got mad at her friends for laughing at me. And as she was helping me pick up my things, she giggled, not bec\r\nause of me but because she also fell in those damn stairs in the library and she felt like shit when people just stared at her so she didn't want the same thing to happen to me. She had such a distinctive laugh, the best one I have ever heard so far. She had to leave before I could even thank her and ask for her name. All I know is that she was wearing the film org lanyard. She might be an Ican because I knew some of the people she was with were from that school. I really want to see this girl again. I have yet to thank her and I really am looking forward to personally meet someone with her kindness. If any of you know her, please tag her. Because I think I just met my soul mate.","December 9, 2018 2:15:33 AM +08"
5,18289,"ok so...\r\n\r\nsince we do well in UAAP Women's volleyball, can the team members post shirtless pics on their Instagram as a service to the THOUSANDS of fans like me who watched their games thoughout this season?????????? I probably spent more than 5k for tickets this season\r\n Hello to all the hot gals in the team. pls tag them here :(#ADMUFreedomWall18235","December 9, 2018 2:01:45 AM +08"
6,18288,Only fans will know this #teamdentistjet,"December 9, 2018 1:53:15 AM +08"
7,18287,mood,"December 9, 2018 1:49:07 AM +08"
8,18286,NaN,NaN
9,18285,u know there's something wrong w the prof when their students ask other classes for notes and resources because their own prof didn't fucking teach them anything,"December 9, 2018 1:09:35 AM +08"


In [5]:
# Convert to list
data = df['Body'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove Other ADMU FW mentions
data = [re.sub('\#ADMUFreedomWall\ \d+', '', sent) for sent in data]
data = [re.sub('\#ADMUFreedomWall\d+', '', sent) for sent in data]

pprint(data[:15])

['there’s this guy who’s been making too much malaswa, dagger stares at me, '
 'even if I never done anything wrong with him. The way he looked at me is '
 'like he’s wants to either beat me up physically or even sexually. he’s too '
 'old to act so irrational. Ateneo pls help make the school a better place '
 'rather than bring it a “malaswa playground”',
 'You call yourself my best friend pero you cant even give 5 mins of your time '
 'i needed someone to talk tapos tangina you said stuff pa to make me feel bad '
 'about myself Get your head out of your fucking ass and stop thinking na '
 'everyone wants to make out with u',
 '#Daddy Jett 🙌',
 'https://www.facebook.com/braveyawtz/videos/1744836578868329/ Its crazy to '
 'realize how much all the attention on the war on drugs has died down. Seeing '
 'this video was reminiscent to how drug cartels would summarily execute '
 'people off the streets. If people are dying left and right, and the notion '
 'of labeling their deaths as poli

In [6]:
# Tokenizing
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['there', 'this', 'guy', 'who', 'been', 'making', 'too', 'much', 'malaswa', 'dagger', 'stares', 'at', 'me', 'even', 'if', 'never', 'done', 'anything', 'wrong', 'with', 'him', 'the', 'way', 'he', 'looked', 'at', 'me', 'is', 'like', 'he', 'wants', 'to', 'either', 'beat', 'me', 'up', 'physically', 'or', 'even', 'sexually', 'he', 'too', 'old', 'to', 'act', 'so', 'irrational', 'ateneo', 'pls', 'help', 'make', 'the', 'school', 'better', 'place', 'rather', 'than', 'bring', 'it', 'malaswa', 'playground']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\Gab\Anaconda3\lib\site-packages\gensim\models\phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['there', 'this', 'guy', 'who', 'been', 'making', 'too', 'much', 'malaswa', 'dagger', 'stares', 'at', 'me', 'even', 'if', 'never', 'done', 'anything', 'wrong', 'with', 'him', 'the', 'way', 'he', 'looked', 'at', 'me', 'is', 'like', 'he', 'wants', 'to', 'either', 'beat', 'me', 'up', 'physically', 'or', 'even', 'sexually', 'he', 'too', 'old', 'to', 'act', 'so', 'irrational', 'ateneo', 'pls', 'help', 'make', 'the', 'school', 'better', 'place', 'rather_than', 'bring', 'it', 'malaswa', 'playground']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    output = [[word for word in simple_preprocess(str(doc)) if (word not in stop_words and word not in fil_stop_words)] for doc in texts]
    return output
    
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Gab\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [10]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['guy', 'make', 'much', 'malaswa', 'dagger', 'stare', 'even', 'never', 'do', 'anything', 'wrong', 'way', 'look', 'want', 'beat', 'physically', 'even', 'sexually', 'old', 'act', 'irrational', 'ateneo', 'help', 'make', 'school', 'good', 'place', 'rather', 'bring', 'malaswa', 'playground']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"game" + 0.021*"like" + 0.019*"big" + 0.018*"final" + 0.018*"watch" + '
  '0.017*"next" + 0.016*"ateneo" + 0.015*"basketball" + 0.013*"admin" + '
  '0.013*"reach"'),
 (1,
  '0.040*"yung" + 0.031*"see_translation" + 0.024*"university" + '
  '0.021*"jesuit" + 0.021*"instead" + 0.018*"paper" + 0.017*"email" + '
  '0.013*"putangina" + 0.013*"dlsu" + 0.012*"step"'),
 (2,
  '0.037*"shit" + 0.035*"grade" + 0.032*"miss" + 0.024*"wonder" + '
  '0.014*"notice" + 0.014*"tangina" + 0.013*"meeting" + 0.012*"fast" + '
  '0.012*"almost" + 0.011*"expect"'),
 (3,
  '0.039*"first" + 0.021*"block" + 0.020*"share" + 0.014*"join" + '
  '0.013*"pretty" + 0.013*"little" + 0.012*"especially" + 0.011*"kailangan" + '
  '0.010*"include" + 0.010*"sobrang"'),
 (4,
  '0.039*"life" + 0.030*"bad" + 0.029*"talk" + 0.024*"always" + 0.015*"time" + '
  '0.015*"wish" + 0.014*"guess" + 0.014*"call" + 0.014*"problem" + '
  '0.013*"never"'),
 (5,
  '0.033*"anyone" + 0.026*"kind" + 0.017*"damn" + 0.012*"finally"

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.139770385795106

Coherence Score:  0.495706615090119


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Gab\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.337883 -0.128836  1       1        22.128910
6     -0.331043 -0.006517  2       1        21.184422
4     -0.059774  0.260073  3       1        7.404541 
0      0.062270 -0.088276  4       1        5.859059 
11    -0.016030  0.230045  5       1        5.457487 
8      0.074369 -0.060230  6       1        5.403565 
5      0.055610 -0.011340  7       1        4.597725 
3      0.068733 -0.019721  8       1        4.482946 
7      0.073694 -0.037172  9       1        4.454253 
13     0.065244 -0.005846  10      1        4.169313 
12     0.086643 -0.043245  11      1        4.107291 
1      0.082239 -0.038055  12      1        3.852636 
2      0.092395 -0.027763  13      1        3.489448 
9      0.083532 -0.023116  14      1        3.408405 , topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
41    Default  213.000000  not           213.000000  30.0000  30.0000
79    Default  310.000000  people        310.000000  29.0000  29.0000
116   Default  188.000000  go            188.000000  28.0000  28.0000
8     Default  151.000000  good          151.000000  27.0000  27.0000
55    Default  155.000000  be            155.000000  26.0000  26.0000
112   Default  130.000000  friend        130.000000  25.0000  25.0000
693   Default  87.000000   life          87.000000   24.0000  24.0000
441   Default  61.000000   start         61.000000   23.0000  23.0000
34    Default  151.000000  feel          151.000000  22.0000  22.0000
343   Default  117.000000  try           117.000000  21.0000  21.0000
42    Default  116.000000  say           116.000000  20.0000  20.0000
122   Default  199.000000  know          199.000000  19.0000  19.0000
46    Default  75.000000   talk          75.000000   18.0000  18.0000
36    Default  207.000000  get           207.000000  17.0000  17.0000
29    Default  64.000000   bad           64.000000   16.0000  16.0000
37    Default  58.000000   give          58.000000   15.0000  15.0000
95    Default  162.000000  would         162.000000  14.0000  14.0000
626   Default  49.000000   first         49.000000   13.0000  13.0000
178   Default  117.000000  s             117.000000  12.0000  12.0000
4954  Default  37.000000   doo_doo       37.000000   11.0000  11.0000
86    Default  101.000000  see           101.000000  10.0000  10.0000
621   Default  38.000000   yung          38.000000   9.0000   9.0000 
135   Default  154.000000  really        154.000000  8.0000   8.0000 
7     Default  131.000000  even          131.000000  7.0000   7.0000 
194   Default  84.000000   person        84.000000   6.0000   6.0000 
13    Default  181.000000  make          181.000000  5.0000   5.0000 
192   Default  93.000000   love          93.000000   4.0000   4.0000 
25    Default  184.000000  want          184.000000  3.0000   3.0000 
184   Default  37.000000   anyone        37.000000   2.0000   2.0000 
50    Default  134.000000  time          134.000000  1.0000   1.0000 
...       ...        ...      ...              ...      ...      ... 
573   Topic14  16.297331   true          16.968969   3.3385  -3.9342 
212   Topic14  12.543685   listen        13.215309   3.3268  -4.1960 
254   Topic14  9.544827    gusto         10.216450   3.3109  -4.4692 
1524  Topic14  9.408029    sayo          10.079650   3.3100  -4.4836 
2288  Topic14  8.617172    option        9.288802    3.3039  -4.5714 
942   Topic14  8.452950    hand          9.124579    3.3025  -4.5907 
221   Topic14  8.428219    random        9.099848    3.3023  -4.5936 
1548  Topic14  7.582474    gago          8.254096    3.2941  -4.6993 
1302  Topic14  7.466125    regis         8.137748    3.2928  -4.7148 
554   Topic14  7.316792    involve       7.988470    3.2911  -4.7350 
2164  Topic14  7.283236    aside         7.954863    3.2907  -4.7396 
560   Topic14

# Using Mallet

In [16]:
mallet_path = 'C:\\Mallet/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=id2word)

In [17]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('feel', 0.05970772442588727),
   ('time', 0.05636743215031315),
   ('people', 0.037578288100208766),
   ('good', 0.03632567849686848),
   ('bad', 0.025052192066805846),
   ('thing', 0.021711899791231733),
   ('stop', 0.019206680584551147),
   ('friend', 0.018789144050104383),
   ('meet', 0.01837160751565762),
   ('leave', 0.017118997912317326)]),
 (1,
  [('ateneo', 0.05763567522086664),
   ('post', 0.04291123264619268),
   ('school', 0.02902818679007152),
   ('student', 0.026503996634413125),
   ('shit', 0.022297013041649136),
   ('atenean', 0.019772822885990745),
   ('hate', 0.01851072780816155),
   ('lose', 0.009676062263357174),
   ('prof', 0.009676062263357174),
   ('university', 0.009255363904080775)]),
 (2,
  [('people', 0.050084175084175085),
   ('talk', 0.03324915824915825),
   ('work', 0.029461279461279462),
   ('thing', 0.027777777777777776),
   ('fuck', 0.023989898989898988),
   ('hard', 0.023148148148148147),
   ('start', 0.022727272727272728),
   ('hope', 0.021885

In [18]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [19]:
# Can take a long time to run.
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [20]:
# Show graph
# limit=40; start=2; step=6;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [21]:
# Print the coherence scores
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [22]:
# Select the model and print the topics
# optimal_model = model_list[1]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.060*"feel" + 0.056*"time" + 0.038*"people" + 0.036*"good" + 0.025*"bad" + '
  '0.022*"thing" + 0.019*"stop" + 0.019*"friend" + 0.018*"meet" + '
  '0.017*"leave"'),
 (1,
  '0.058*"ateneo" + 0.043*"post" + 0.029*"school" + 0.027*"student" + '
  '0.022*"shit" + 0.020*"atenean" + 0.019*"hate" + 0.010*"lose" + 0.010*"prof" '
  '+ 0.009*"university"'),
 (2,
  '0.050*"people" + 0.033*"talk" + 0.029*"work" + 0.028*"thing" + 0.024*"fuck" '
  '+ 0.023*"hard" + 0.023*"start" + 0.022*"hope" + 0.017*"make" + '
  '0.016*"understand"'),
 (3,
  '0.035*"class" + 0.025*"miss" + 0.018*"back" + 0.016*"sad" + 0.016*"year" + '
  '0.015*"day" + 0.015*"week" + 0.014*"sem" + 0.014*"end" + 0.013*"doo_doo"'),
 (4,
  '0.054*"make" + 0.038*"friend" + 0.033*"person" + 0.033*"people" + '
  '0.027*"love" + 0.027*"life" + 0.019*"group" + 0.015*"world" + 0.014*"part" '
  '+ 0.013*"stay"'),
 (5,
  '0.030*"game" + 0.027*"yung" + 0.023*"crush" + 0.023*"call" + '
  '0.022*"basketball" + 0.021*"watch" + 0.020*"fin

## Most dominant topic in a document

In [23]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [24]:
# Show dominant topic in a document
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.1566,"people, talk, work, thing, fuck, hard, start, hope, make, understand","there’s this guy who’s been making too much malaswa, dagger stares at me, even if I never done anything wrong with him. The way he looked at me is like he’s wants to either beat me up physically or even sexually. he’s too old to act so irrational. Ateneo pls help make the school a better place rather than bring it a “malaswa playground”"
1,1,0.0,0.1679,"feel, time, people, good, bad, thing, stop, friend, meet, leave",You call yourself my best friend pero you cant even give 5 mins of your time i needed someone to talk tapos tangina you said stuff pa to make me feel bad about myself Get your head out of your fucking ass and stop thinking na everyone wants to make out with u
2,2,5.0,0.1394,"game, yung, crush, call, basketball, watch, final, see_translation, wrong, uaap",#Daddy Jett 🙌
3,3,4.0,0.2154,"make, friend, person, people, love, life, group, world, part, stay","https://www.facebook.com/braveyawtz/videos/1744836578868329/ Its crazy to realize how much all the attention on the war on drugs has died down. Seeing this video was reminiscent to how drug cartels would summarily execute people off the streets. If people are dying left and right, and the notion of labeling their deaths as police self defence, or that the victims fought back, and leaving it at that, then no one is safe to this kind of cruelty. Tomorrow, the very people being left to die on the streets just might be me and you. Unless of course youre of a rich and/or influential family. Move along, I guess this shouldnt concern you."
4,4,0.0,0.2674,"feel, time, people, good, bad, thing, stop, friend, meet, leave","I saw this girl last thursday. She was hanging out with a bunch of Icans and while I was going down the stairs, I tripped. All of them laughed except for one, she helped me and even asked if I was okay. I was so flustered and just stared at her face. She had the most amazing smile and I couldnt even utter a word. It was funny because she actually got mad at her friends for laughing at me. And as she was helping me pick up my things, she giggled, not bec ause of me but because she also fell in those damn stairs in the library and she felt like shit when people just stared at her so she didnt want the same thing to happen to me. She had such a distinctive laugh, the best one I have ever heard so far. She had to leave before I could even thank her and ask for her name. All I know is that she was wearing the film org lanyard. She might be an Ican because I knew some of the people she was with were from that school. I really want to see this girl again. I have yet to thank her and I really am looking forward to personally meet someone with her kindness. If any of you know her, please tag her. Because I think I just met my soul mate."
5,5,5.0,0.2226,"game, yung, crush, call, basketball, watch, final, see_translation, wrong, uaap","ok so... since we do well in UAAP Womens volleyball, can the team members post shirtless pics on their Instagram as a service to the THOUSANDS of fans like me who watched their games thoughout this season?????????? I probably spent more than 5k for tickets this season Hello to all the hot gals in the team. pls tag them here :("
6,6,5.0,0.1394,"game, yung, crush, call, basketball, watch, final, see_translation, wrong, uaap",Only fans will know this #teamdentistjet
7,7,1.0,0.1400,"ateneo, post, school, student, shit, atenean, hate, lose, prof, university",mood
8,8,1.0,0.1940,"ateneo, post, school, student, shit, atenean, hate, lose, prof, university",u know theres something wrong w the prof when their students ask other classes for notes and resources because their own prof didnt fucking teach them anything
9,9,4.0,0.1654,"make, friend, person, people, love, life, group, world, part, stay","Im not enough for anyone. Best friend/ almost/ safe choice. That is all I have ever been and will probabl

## Topic Interpretation

In [39]:
counter = 0
topic_df = pd.DataFrame()
curr_topics = []
while True:
    try:
        curr_topic = optimal_model.show_topic(counter)
        curr_topics.append([topic[0] for topic in curr_topic])
        counter += 1
    except:
        break
topic_df['Topic Keywords'] = curr_topics

interpretations = [i for i in range(8)]
interpretations[0] = "Social Relations"
interpretations[1] = "University"
interpretations[2] = "?"
interpretations[3] = "Academics"
interpretations[4] = "Social Relations"
interpretations[5] = "Sports"
interpretations[6] = "Love and Relationships"
interpretations[7] = "?"

topic_df['Interpretation'] = interpretations
topic_df

,Topic Keywords,Interpretation
0,"[feel, time, people, good, bad, thing, stop, friend, meet, leave]",Social Relations
1,"[ateneo, post, school, student, shit, atenean, hate, lose, prof, university]",University
2,"[people, talk, work, thing, fuck, hard, start, hope, make, understand]",?
3,"[class, miss, back, sad, year, day, week, sem, end, doo_doo]",Academics
4,"[make, friend, person, people, love, life, group, world, part, stay]",Social Relations
5,"[game, yung, crush, call, basketball, watch, final, see_translation, wrong, uaap]",Sports
6,"[guy, girl, love, admu, play, study, cute, wanna, hear, country]",Love and Relationships
7,"[good, give, car, big, change, high, date, improve, find, end]",?


## Most dominant document in a topic

In [26]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.5491,"feel, time, people, good, bad, thing, stop, friend, meet, leave","You know Im honestly so frustrated about Grab Coco. I miss Regis Coco before it was just overflowing with Grab Drivers. Of course nothing is going to change because people can have Coco wherever they are, and Coco is earning so much more because the number of customers has skyrocketed. But Ive been going to Regis Coco because I love the place and I love hanging out with friends over there. It was a nice place to relax and have a good time. The ambiance was lively but not exhausting. It was homely but not boring. I have had wonderful memories in every seat, and have had laughs with some of the employees even. But the Grab drivers just ruin it. The line is so long, I could stand in line for 30 minutes. Even when I do get to order, I could wait an hour in my seat before I get my drink. Why? Because there are 10 Grab drivers, each ordering multiple drinks. Coco wasnt always like this. Of course there was usually a line, and of course there was always a wait time. But in the past few months, the lines have gotten unbearably long, and the waiting time has become ridiculous. Customers who still go there in-person suffer. There are chairs and table there because its also a sit-down place; part of Coco is the store, not just the drink. Its not just a hole in the wall that pumps out drinks to people. Im letting out my frustrations, but I know nothings gonna change. In fact, it could get even worse. Cocos earning, Grab drivers will have a consistent source of income, and people are comfortable having the Coco come to them. But for someone who has loved Coco for years, with part of that experience coming from sitting in the store and enjoying a few hours IN the physical store of Coco, I miss those days. I wish those days would come back, but I should know better. The old Coco is dead."
1,1.0,0.4751,"ateneo, post, school, student, shit, atenean, hate, lose, prof, university","Note: 99% of Ateneans dont agree with the ""many UP students being in Regis are annoying"" post. But Im not here for that, this is a university rank discussion post. The tweet that became popular said that Ateneo was just a 2nd rate University compared to U.P. https://mobile.twitter.com/piodu…/status/1044147879479111680 I know its a joke tweet but it reminded me of that age old discussion of University rankings. In the end, the general rank of your University is a far less important than the rank of your specific course. A minority of U.P students like to brag towards Ateneans, likewise a minority of Ateneans brag against DLSU and so on...but the university rankings are meaningless when you check the board exam passing rate of your specific course. The top 3 medical schools in the country arent even from the big 4. So course rank is a lot more important than school rank. Ofcourse, dont forget about mental health though."
2,2.0,0.6466,"people, talk, work, thing, fuck, hard, start, hope, make, understand","Dont just go to guidance, sanggu or ADSA right away. Talk to a lawyer first. This is unsolicited advice to seek legal advice to rape victims like in The guidance, sanggu or ADSA may or may not represent your interests, but your lawyer would. The guidance, sanggu or ADSA might (we wont really know for sure until AFTER THE FACT) be more interested in bringing the rapist to justice than giving you justice. You might think these two concepts ""bringing the perpetrator to justice"" and ""giving the victim justice"" are identical. If they are, then you can ignore this post. See 13 Reasons Why or any legal drama where rape victims are cross-examined furiously by defense attorneys. (For non-fictional examples, I wouldnt know of any that are relevant or relatable to non-lawyers. Lets hope 13 Reasons Why or those legal dramas are as realistic as they need to be here for this purpose) Prosecuting rapists is hard because our society values of privacy over se